In [ ]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
def train_network():
    notes, chords = get_notes()
    # get amount of pitch names
    flattened_chord = set()
    for chord in chords: flattened_chord.update(set(chord))
    n_vocab = len(  set(notes).union( flattened_chord )  )
    network_input, note_output, chord_output = prepare_sequences(notes, chords, n_vocab)
    
    note_model = create_network(network_input, n_vocab)
    chord_model = create_network(network_input, n_vocab)
    
    history_note = train(note_model, network_input, note_output, typeof="note")
    history_chord = train(chord_model, network_input, chord_output, typeof="chord")
    
    return history_note, history_chord

In [ ]:
def get_notes():
    notes = []
    chords = []
    i=0
    for file in glob.glob("lmd_matched/**/*.mid", recursive=True):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                chords.append([str(note.pitch) for note in element])
        if i == 5: break
        i+=1
    with open('data/notes', 'wb') as filepath:
        pickle.dump((notes, chords), filepath)

    return notes, chords

In [ ]:
def prepare_sequences(notes, chords, n_vocab):
    sequence_length = 100
    # get all pitch names
    flattened_chord = set()
    for chord in chords: flattened_chord.update(set(chord))
    pitchnames = sorted(set(item for item in notes).union(flattened_chord))
#     pitchnames = sorted(set(item for item in notes))
     #map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    note_output = []
    chord_output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = []
        for j in range(i, i+ sequence_length):
            sequence_in += [notes[j]] + [chords[j][l] for l in range(len(chords[j]))]
            
        sequence_out_notes = notes[i + sequence_length]
        sequence_out_chords = chords[i + sequence_length]
        
        network_input.append([note_to_int[char] for char in sequence_in])
        note_output.append(note_to_int[sequence_out_notes])
        chord_output.append([note_to_int[char] for char in sequence_out_chords])

    n_patterns = len(network_input)

    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize
    network_input = network_input / float(n_vocab)
    note_output = np_utils.to_categorical(note_output)
    chord_output = np_utils.to_categorical([chord_output])
    
    return (network_input, note_output, chord_output)

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


In [ ]:
def train(model, network_input, network_output, typeof):
    """ train the neural network """
    if typeof == "note":
        filepath = "checkpoints_lakh/note-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
    else:
        filepath = "checkpoints_lakh/chord-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    history = model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list)
    return history

In [ ]:
history_note, history_chord = train_network()

In [ ]:
print(history_note.history.keys())
# summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for loss
plt.plot(history_note.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        (notes, chords) = pickle.load(filepath)
    
    flattened_chord = set()
    for chord in chords: flattened_chord.update(set(chord))
    n_vocab = len(  set(notes).union( flattened_chord )  )
    pitchnames = sorted(set(item for item in notes).union(flattened_chord))
    # Get all pitch names
#     pitchnames = sorted(set(item for item in notes))
#     # Get all pitch names
#     n_vocab = len(set(notes))

    network_input, note_output, chord_output = prepare_sequences(notes, chords, n_vocab)
    note_model = create_network(network_input, n_vocab, typeof="note")
    chord_model = create_network(network_input, n_vocab, typeof="chord")
    
    prediction_output = generate_notes(note_model, chord_model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    if typeof == "note":
        model.load_weights('checkpoints_lakh/note-weights-improvement-54-1.2406-bigger.hdf5')
    else:
       model.load_weights('checkpoints_lakh/chord-weights-improvement-54-1.2406-bigger.hdf5')
    return model

In [ ]:
def generate_notes(note_model, chord_model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        note_prediction = note_model.predict(prediction_input, verbose=0)

        index_note = np.argmax(note_prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        chord_prediction = chord_model.predict(prediction_input, verbose=0)
        index_chord = np.argmax(chord_prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern.append(index)
        pattern.append(index_chord)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_lakh.mid')

In [ ]:
generate()